In [ ]:
### QUESTIONS

# how to adjust for lease start time? inflation plays into this.
    # do we group in years of 2, 3, 4, 5?
    # match this against economic data
# construction year might turn into 'age'
# buildingrating id is categorical--cluster analysis may be needed...
# use submarket vs CBSID?
# calculate STD for each region in terms of rent
    #which submarkets have the most variance? within AND outside of buildingratingid
# link population to this?

In [19]:
# data processing
import numpy as np
import pandas as pd
from datetime import date, timedelta
import datetime as dt
import pyodbc 

# modeling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(17325)

In [2]:
def query_costar(query):
    ''' query from Costar Database'''

    conn = pyodbc.connect('Driver={SQL Server};'
                        'Server=DESKTOP-V8FUEV1\SQL12345;'
                        'Database=Costar;'
                        'Trusted_Connection=yes;')
    cursor = conn.cursor()

    df = pd.read_sql_query(query, conn)

    return  df

In [6]:
def merge_lease_geo(how):
    '''pull sql server tables into DFs and merge, drop cols'''

    q1 = 'select * from dbo.merged_lease'
    df1 = query_costar(q1)

    q2 = 'select * from dbo.submarket_geo'
    df2 = query_costar(q2)

    df = pd.merge(df1, df2, 
                        how = how, 
                        left_on = 'leasedealid',
                        right_on = 'LeaseDealID')

    drop_cols = ['estimatedrent_x']

    df.drop(columns = drop_cols, axis = 1, inplace=True)

    return df

df = merge_lease_geo('inner')
df.shape

,leasedealid,propertyid,propertytypeid,locationoccupancyid,servicetypeid,sqftmin,sqftmax,dateonmarket,dateoffmarket,daysonmarket,...,cbsaid,cbsa_cities,cbsa_states,constructionyear,buildingratingid,LeaseDealID,estimatedrent_y,researchmarketname,submarketname,zip
0,50017763,487493.0,5.0,50017763.0,None,None,None,None,None,None,...,17460.0,Cleveland-Elyria,OH,1959.0,2.0,50017763,11.861200,Cleveland,Midtown,44114
1,60003305,164525.0,5.0,60003305.0,None,None,None,None,None,None,...,35084.0,Newark,NJ-PA,1992.0,4.0,60003305,32.756071,Northern New Jersey,Newark,7102
2,110129344,380833.0,5.0,110710987.0,7.0,9237.0,9237.0,2001-09-30,2005-12-31,1553.0,...,36740.0,Orlando-Kissimmee-Sanford,FL,1969.0,4.0,110129344,24.994807,Orlando,Downtown Orlando,32801
3,10616328,137301.0,5.0,10616328.0,7.0,None,None,None,None,None,...,47900.0,Washington-Arlington-Alexandria,DC-VA-MD-WV,1986.0,4.0,10616328,27.716402,"Washington, DC",Clarendon/Courthouse,22201
4,110506515,224664.0,5.0,111775716.0,7.0,5360.0,5360.0,2004-06-30,2006-12-31,914.0,...,38060.0,Phoenix-Mesa-Scottsdale,AZ,1986.0,4.0,110506515,26.411667,Phoenix,Paradise Valley,85028


In [13]:
#extract year from lease 'fromdate'
df['year'] = pd.DatetimeIndex(df['fromdate']).year

In [35]:
#calculate building age (years) from 'constructionyear'
df['building_age'] = today.year - df['constructionyear']

In [37]:
def pct_null():
    '''% of null values for pca analysis'''
    
    null_pct = df.isna().sum()/df.shape[0]
    print(null_pct)

lst = pct_null()

leasedealid                          0.000000
propertyid                           0.013111
propertytypeid                       0.013111
locationoccupancyid                  0.013111
servicetypeid                        0.161845
sqftmin                              0.369553
sqftmax                              0.369553
dateonmarket                         0.369553
dateoffmarket                        0.369553
daysonmarket                         0.369553
renewal                              0.798051
actualvacancy                        0.369553
rateactual                           0.774229
rba                                  0.015003
fromdate                             0.013111
todate                               0.290424
leaseterminmonths                    0.013111
leasesigndate                        0.013113
leaseexpirationdate                  0.013111
tenantimprovementallowancepersqft    0.976298
freemonths                           0.852551
cbsaid                            

In [40]:
pca_cols = ['sqftmin', 'daysonmarket', 'buildingratingid', 
            'estimatedrent_y', 'rba','year', 
            'leaseterminmonths', 'building_age']
df2 = df[pca_cols]

df2.to_csv(r'c:\temp\lease_merged_trim.csv', index = False)

Index(['sqftmin', 'daysonmarket', 'buildingratingid', 'estimatedrent_y', 'rba',
       'year', 'leaseterminmonths', 'building_age'],
      dtype='object')

In [41]:
### use export 'lease_merged_trim.csv' file for JMP pca analysis.